This document outlines the process for importing data and basic data cleaning that I followed for this project. Initial cleaning examples are done on a smaller dataframe to save processing time.

## Importing data into my database

#### Importing the MTA Turnstile data:

Step 1: Run `python get_mta_turnstiles.py "(19|20|21|22)"` in terminal to get the turnstile data. 

I wanted all of the data from 2019-2021 in my database, so I included 22 in the regex to make sure I got everything from 2021 (since the last few days of 2021 were included in the first upload in 2022). I could have been more specific here, but this worked for my purposes. 

#### Importing the MTA Fare data:

Note: due to formatting differences in the MTA Fare data, I had to modify the get_mta.py file and split it into get_mta_fares.py and get_mta_fares_old.py.

Step 1: Run `python get_mta_fares_new.py "(20|21|22)"` to get the bulk of the new format of MTA fare data.

Step 2: Run `python get_mta_fares_old.py "19"` to get the remaining MTA fare data for 2019.

#### Importing the MTA Turnstiles Remote-Complex Lookup:

I found a hand-built resource from Chris Whong that matches MTA station data found at `http://web.mta.info/developers/data/nyct/subway/Stations.csv` to the fields in the turnstile dataset. 

In [221]:
reset -fs

In [222]:
import pandas as pd
import datetime as dt
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///database/mta_data.db')

In [223]:
remote_lookup_df = pd.read_csv('database/mta_remote_lookup.csv')
remote_lookup_df.columns = [column.strip().upper() for column in remote_lookup_df.columns]
remote_lookup_df.head()

,REMOTE,BOOTH,COMPLEX_ID,STATION,LINE_NAME,DIVISION
0,R001,A060,635.0,WHITEHALL ST,R1,BMT
1,R001,A058,635.0,WHITEHALL ST,R1,BMT
2,R001,R101S,635.0,SOUTH FERRY,R1,IRT
3,R002,A077,628.0,FULTON ST,ACJZ2345,BMT
4,R002,A081,628.0,FULTON ST,ACJZ2345,BMT


In [224]:
# strip whitespace from entries - just to be on the safe side
lookup = (['REMOTE', 
             'BOOTH', 
             'COMPLEX_ID', 
             'STATION', 
             'LINE_NAME', 
             'DIVISION'])
(remote_lookup_df[lookup].str.strip() for column in lookup)
remote_lookup_df.head()

,REMOTE,BOOTH,COMPLEX_ID,STATION,LINE_NAME,DIVISION
0,R001,A060,635.0,WHITEHALL ST,R1,BMT
1,R001,A058,635.0,WHITEHALL ST,R1,BMT
2,R001,R101S,635.0,SOUTH FERRY,R1,IRT
3,R002,A077,628.0,FULTON ST,ACJZ2345,BMT
4,R002,A081,628.0,FULTON ST,ACJZ2345,BMT


In [225]:
remote_lookup_df.COMPLEX_ID

0      635.0
1      635.0
2      635.0
3      628.0
4      628.0
       ...  
771    477.0
772    476.0
773    476.0
774    475.0
775    475.0
Name: COMPLEX_ID, Length: 776, dtype: float64

In [277]:
remote_lookup_df['REMOTE'] = remote_lookup_df['REMOTE'].astype(object)
remote_lookup_df['BOOTH'] = remote_lookup_df['BOOTH'].astype(object)
remote_lookup_df['COMPLEX_ID'] = remote_lookup_df['COMPLEX_ID'].astype(object)
remote_lookup_df['STATION'] = remote_lookup_df['STATION'].astype(object)
remote_lookup_df['LINE_NAME'] = remote_lookup_df['LINE_NAME'].astype(object)
remote_lookup_df['DIVISION'] = remote_lookup_df['DIVISION'].astype(object)
remote_lookup_df.REMOTE

0      R001
1      R001
2      R001
3      R002
4      R002
       ... 
771    R570
772    R571
773    R571
774    R572
775    R572
Name: REMOTE, Length: 776, dtype: object

In [278]:
#remote_lookup_df.to_sql('remote_lookup',engine)

## Processing Fare Data

In order to get one table containing all fare types, including discontinued ones, I ran this:

In [396]:
fare_data_df = pd.read_sql('''SELECT [REMOTE], [STATION], [START_DATE], [END_DATE], [FF], [SEN/DIS], 
                            [7-D AFAS UNL], [30-D AFAS/RMF UNL], [JOINT RR TKT], [7-D UNL], [30-D UNL], 
                            [14-D RFM UNL], [1-D UNL], [14-D UNL], [7D-XBUS PASS], [TCMC], [RF 2 TRIP], 
                            [RR UNL NO TRADE], [TCMC ANNUAL MC], [MR EZPAY EXP], [MR EZPAY UNL], 
                            [PATH 2-T], [AIRTRAIN FF], [AIRTRAIN 30-D], [AIRTRAIN 10-T], 
                            [AIRTRAIN MTHLY], [STUDENTS], [NICE 2-T], [CUNY-120],[CUNY-60], 
                            '0' as [FF VALUE], '0' as [FF 7-DAY], '0' as [FF 30-DAY]
                         FROM mta_fare_data_old
                         UNION ALL
                         SELECT [REMOTE], [STATION], [START_DATE], [END_DATE], [FF], [SEN/DIS], 
                         [7-D AFAS UNL], [30-D AFAS/RMF UNL], [JOINT RR TKT], [7-D UNL], [30-D UNL], 
                         '0' as [14-D RFM UNL], '0' as [1-D UNL], '0' as [14-D UNL], 
                         [7D-XBUS PASS], [TCMC], [RF 2 TRIP], [RR UNL NO TRADE], [TCMC ANNUAL MC], 
                         [MR EZPAY EXP], [MR EZPAY UNL], [PATH 2-T], [AIRTRAIN FF], [AIRTRAIN 30-D], 
                         [AIRTRAIN 10-T], [AIRTRAIN MTHLY], [STUDENTS], [NICE 2-T], [CUNY-120], [CUNY-60], 
                         [FF VALUE], [FF 7-DAY], [FF 30-DAY]
                         FROM mta_fare_data_new;''', engine)
fare_data_df.head()

,REMOTE,STATION,START_DATE,END_DATE,FF,SEN/DIS,7-D AFAS UNL,30-D AFAS/RMF UNL,JOINT RR TKT,7-D UNL,...,AIRTRAIN 30-D,AIRTRAIN 10-T,AIRTRAIN MTHLY,STUDENTS,NICE 2-T,CUNY-120,CUNY-60,FF VALUE,FF 7-DAY,FF 30-DAY
0,R001,WHITEHALL STREET,2019-03-23 00:00:00,2019-03-29 00:00:00,66184,3108,460,1143,227,32448,...,0,0,0,3512,0,858,0,0,0,0
1,R003,CYPRESS HILLS,2019-03-23 00:00:00,2019-03-29 00:00:00,2653,181,8,44,0,2078,...,0,0,0,897,0,81,0,0,0,0
2,R004,75TH STREET & ELDERTS LANE,2019-03-23 00:00:00,2019-03-29 00:00:00,5952,390,42,115,0,4195,...,0,0,0,3138,0,96,0,0,0,0
3,R005,85TH STREET & FOREST PKWAY,2019-03-23 00:00:00,2019-03-29 00:00:00,7000,564,40,144,1,4220,...,0,0,0,1666,0,165,0,0,0,0
4,R006,WOODHAVEN BOULEVARD,2019-03-23 00:00:00,2019-03-29 00:00:00,6712,481,60,168,0,6019,...,0,0,0,2120,0,178,0,0,0,0


Note that this query creates new columns in each of the tables to match the missing fare types from the other and sets the values to zero. The UNION ALL command is used to combine into one table. The SQL query is pretty clunky, but it gets the job done. I then strip the trailing whitespace from the entries in the STATION column and add the table to my database:

In [397]:
fare_data_df['STATION'] = fare_data_df['STATION'].str.strip()
fare_data_df.columns

Index(['REMOTE', 'STATION', 'START_DATE', 'END_DATE', 'FF', 'SEN/DIS',
       '7-D AFAS UNL', '30-D AFAS/RMF UNL', 'JOINT RR TKT', '7-D UNL',
       '30-D UNL', '14-D RFM UNL', '1-D UNL', '14-D UNL', '7D-XBUS PASS',
       'TCMC', 'RF 2 TRIP', 'RR UNL NO TRADE', 'TCMC ANNUAL MC',
       'MR EZPAY EXP', 'MR EZPAY UNL', 'PATH 2-T', 'AIRTRAIN FF',
       'AIRTRAIN 30-D', 'AIRTRAIN 10-T', 'AIRTRAIN MTHLY', 'STUDENTS',
       'NICE 2-T', 'CUNY-120', 'CUNY-60', 'FF VALUE', 'FF 7-DAY', 'FF 30-DAY'],
      dtype='object')

In [398]:
#fare_data_df.to_sql('fare_data_total',engine)


In [399]:
fare_data_df.shape

(76063, 33)

In [400]:
# remove bus data
#bus = ['R553', 'R554', 'R555', 'R556', 'R557', 'R558', 'R559', 'R560', 'R561', 'R562', 'R563', 'R564', 'R565', 'R566', 'R567', 'R568', 'R569', 'R573', 'R574', 'R575'] 

#fare_data_df = fare_data_df[(fare_data_df.REMOTE != [remove])] 

#fare_data_df = fare_data_df[~fare_data_df.REMOTE.isin(bus)]
#fare_data_df.shape

Then I insert the values from the `remote_lookup_df` into the `fare_data_df` for ease of reference:

In [401]:
fare_data_df.rename(columns={'STATION': 'STOP'}, inplace=True)
fare_data_df.shape

(76063, 33)

In [402]:
#fare_data_df.insert(1,'BOOTH',0)
#fare_data_df.insert(2,'COMPLEX_ID',0)
#fare_data_df.insert(3,'STATION',0)
#fare_data_df.insert(5,'LINE_NAME',0)
#fare_data_df.insert(6,'DIVISION',0)
#fare_data_df.shape

In [403]:
fare_data_df = fare_data_df.merge(remote_lookup_df,on='REMOTE',how="left")
fare_data_df.columns

Index(['REMOTE', 'STOP', 'START_DATE', 'END_DATE', 'FF', 'SEN/DIS',
       '7-D AFAS UNL', '30-D AFAS/RMF UNL', 'JOINT RR TKT', '7-D UNL',
       '30-D UNL', '14-D RFM UNL', '1-D UNL', '14-D UNL', '7D-XBUS PASS',
       'TCMC', 'RF 2 TRIP', 'RR UNL NO TRADE', 'TCMC ANNUAL MC',
       'MR EZPAY EXP', 'MR EZPAY UNL', 'PATH 2-T', 'AIRTRAIN FF',
       'AIRTRAIN 30-D', 'AIRTRAIN 10-T', 'AIRTRAIN MTHLY', 'STUDENTS',
       'NICE 2-T', 'CUNY-120', 'CUNY-60', 'FF VALUE', 'FF 7-DAY', 'FF 30-DAY',
       'BOOTH', 'COMPLEX_ID', 'STATION', 'LINE_NAME', 'DIVISION'],
      dtype='object')

### Computing Commuter Percentage

I want to know the percentage of likely commuter traffic for each station in this dataset. First I sum the total swipes across all categories:

In [404]:
fare_data_df['TOTAL'] = fare_data_df.iloc[:,4:33].sum(axis=1)
fare_data_df['TOTAL']

/var/folders/_g/0w700wks7s1f9mzvhbcn7vvc0000gn/T/ipykernel_55257/1067136374.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



0         159904
1         159904
2         159904
3           7318
4          17766
           ...  
121871    160085
121872    116168
121873    116168
121874      1026
121875       711
Name: TOTAL, Length: 121876, dtype: int64

Now I want to sum only those columns that I want to use as proxy for commuters. I look at the MTA Field Descriptions on the Fare Data page and compare them to the columns in `fare_data_df`:

In [405]:
fare_data_df.columns

Index(['REMOTE', 'STOP', 'START_DATE', 'END_DATE', 'FF', 'SEN/DIS',
       '7-D AFAS UNL', '30-D AFAS/RMF UNL', 'JOINT RR TKT', '7-D UNL',
       '30-D UNL', '14-D RFM UNL', '1-D UNL', '14-D UNL', '7D-XBUS PASS',
       'TCMC', 'RF 2 TRIP', 'RR UNL NO TRADE', 'TCMC ANNUAL MC',
       'MR EZPAY EXP', 'MR EZPAY UNL', 'PATH 2-T', 'AIRTRAIN FF',
       'AIRTRAIN 30-D', 'AIRTRAIN 10-T', 'AIRTRAIN MTHLY', 'STUDENTS',
       'NICE 2-T', 'CUNY-120', 'CUNY-60', 'FF VALUE', 'FF 7-DAY', 'FF 30-DAY',
       'BOOTH', 'COMPLEX_ID', 'STATION', 'LINE_NAME', 'DIVISION', 'TOTAL'],
      dtype='object')

Field Description for the columns:\
FF = full fare\
SEN/DIS = senior citizen/disabled\
7-D AFAS UNL = 7-day unlimited ADA farecard access system\
30-D AFAS/RFM UNL = 30-day unlimited ADA farecard access system/reduced fare media\
JOINT RR TKT = Joint railroad ticket\
7-D UNL = 7-day unlimited\
30-D UNL = 30-day unlimited\
14-D RFM UNL = 14-day reduced fare media unlimited\
1-D UNL = one day unlimited\
14-D UNL = 14-day unlimited\
7D-XBUS PASS = 7-day express bus pass\
TCMC = Transit check metrocard\
RF 2 TRIP = reduced fare 2 trip\
RR UNL NO TRADE = Railroad unlimited no trade\
TCMC ANNUAL MC = Transit check annual metrocard\
MR EZPAY EXP = Mail and Ride EZ-pay express\
MR EZPAY UNL = Mail and Ride EZ-pay unlimited\
PATH 2-T = PATH train 2-trip\
AIRTRAIN FF = Airtrain full fare\
AIRTRAIN 30-D = Airtrain 30-day\
AIRTRAIN 10-T = Airtrain 10-trip\
AIRTRAIN MTHLY = Airtrain monthly\
STUDENTS = School-age students (non-college)\
NICE 2-T = ?? two-trip card\
CUNY-120 = CUNY community college 120-day\
CUNY-60 = CUNY community college 60-day\
FF VALUE = Fair fares NYC (half price for low-income NYC residents)\
FF 7-DAY = Fair Fares 7-Day\
FF 30-DAY = Fair Fares 30-Day

Based on the field descriptions for the columns, I create a new COMMUTERS column, which includes all student, ADA, and fare reductions for high-value trips or long durations (14-day, monthly, annual). 

I don't include EZ-pay, bus or rail/train cards except for the Airtrain 30-day, 10-trip and monthly cards. (Those cards are likely used by airport workers.) 

I also exclude full fare and one- and two-day unlimited cards, as those are more likely used by visitors. 

In [406]:
commuters = (['SEN/DIS', 
             '7-D AFAS UNL', 
             '30-D AFAS/RMF UNL', 
             '7-D UNL', 
             '30-D UNL', 
             '14-D RFM UNL', 
             '14-D UNL', 
             'TCMC', 
             'TCMC ANNUAL MC', 
             'AIRTRAIN 30-D', 
             'AIRTRAIN 10-T', 
             'AIRTRAIN MTHLY',
             'STUDENTS', 
             'CUNY-120', 
             'CUNY-60', 
             'FF VALUE', 
             'FF 7-DAY', 
             'FF 30-DAY'])
fare_data_df['COMMUTERS'] = fare_data_df[commuters].sum(axis=1)
fare_data_df['COMMUTERS']             

/var/folders/_g/0w700wks7s1f9mzvhbcn7vvc0000gn/T/ipykernel_55257/2411711674.py:19: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



0         86940
1         86940
2         86940
3          4526
4         11426
          ...  
121871    94826
121872    74190
121873    74190
121874     1026
121875      711
Name: COMMUTERS, Length: 121876, dtype: int64

In [407]:
fare_data_df.sort_values(['START_DATE'], inplace = True)
fare_data_df.head()

,REMOTE,STOP,START_DATE,END_DATE,FF,SEN/DIS,7-D AFAS UNL,30-D AFAS/RMF UNL,JOINT RR TKT,7-D UNL,...,FF VALUE,FF 7-DAY,FF 30-DAY,BOOTH,COMPLEX_ID,STATION,LINE_NAME,DIVISION,TOTAL,COMMUTERS
10551,R111,23RD STREET & 8TH AVENUE,2018-12-22 00:00:00,2018-12-28 00:00:00,29364,3235,278,911,21,16113,...,0,0,0,N075,165.0,23 ST,CE,IND,71175,38969
10621,R153,UTICA AVE-STUYVESANT AVE,2018-12-22 00:00:00,2018-12-28 00:00:00,22364,1065,356,427,5,20643,...,0,0,0,N120A,181.0,UTICA AVE,AC,IND,62900,39073
10620,R153,UTICA AVE-STUYVESANT AVE,2018-12-22 00:00:00,2018-12-28 00:00:00,22364,1065,356,427,5,20643,...,0,0,0,N120,181.0,UTICA AVE,AC,IND,62900,39073
10619,R152,ROCKAWAY PKWY-GLENWOOD RD,2018-12-22 00:00:00,2018-12-28 00:00:00,15195,785,261,413,8,12779,...,0,0,0,H041,138.0,ROCKAWAY PKY,L,BMT,40421,24378
10618,R151,STILLWELL AVE-CONEY ISLAND,2018-12-22 00:00:00,2018-12-28 00:00:00,18963,1545,268,1150,1,12917,...,0,0,0,G009,58.0,STILLWELL AVE,DFNQ,BMT,49136,29124


### Visualizing the commuter percentage at a station

Now I take a look at the commuter percentage for one station, to get a visual sense of the pattern from 2019-2021.

In [408]:
commuters_df = (fare_data_df
                        .groupby(['BOOTH', 'STATION', 'END_DATE', 'TOTAL', 'COMMUTERS'],as_index=False)
                        .TOTAL.first())
commuters_df.shape

(119813, 5)

To get the station totals, I need to aggregate:

In [409]:
station_weekly_totals = (commuters_df
                     .groupby(['STATION', 'END_DATE'])
                     [['COMMUTERS', 'TOTAL']].sum().reset_index())

station_weekly_totals.shape

(60964, 4)

I then divide the COMMUTERS column by the TOTAL column to get the percentage of likely commuters:

In [410]:
station_weekly_totals['COMMUTER_PERCENT'] = (station_weekly_totals['COMMUTERS']
                                 .div(station_weekly_totals['TOTAL']).multiply(100))
station_weekly_totals.shape

(60964, 5)

Now I can plot a station's total swipes and likely commuter percentage over 2019-2021:

In [416]:
# data
commuters_times_sq = (station_weekly_totals[station_weekly_totals['STATION'] == 
                                             '42 ST-TIMES SQ'].copy()) #using mask to segment df
# plot
fig = (px.line(commuters_times_sq, 
               x='END_DATE', 
               y=commuters_times_sq.columns[2:4], 
               hover_data=['COMMUTER_PERCENT']))

# Change the layout
fig.update_layout(title_text='MTA Ridership at Times Square from 2019-2021', 
                  legend=dict(
                      title=None)) 
fig.show()

## Turnstile Data Cleaning

To save processing time, I've run a `processing.py` python script on the full dataset from 2019-2021. Below are examples of the methods and functions run on smaller sets of the data. I essentially followed the process outlined in the pair programming exercises.

In [417]:
turnstiles_df = pd.read_sql('SELECT * FROM mta_turnstile_data WHERE [DATE] LIKE "12/%/2019"', engine)
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,03:00:00,REGULAR,7324295,2482512
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,07:00:00,REGULAR,7324305,2482523
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,11:00:00,REGULAR,7324371,2482594
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,15:00:00,REGULAR,7324587,2482647
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,19:00:00,REGULAR,7324963,2482713


In [418]:
# Create a single datetime column
import datetime

turnstiles_df['DATE_TIME'] = pd.to_datetime(turnstiles_df.DATE + ' ' + turnstiles_df.TIME, 
                                            format='%m/%d/%Y %H:%M:%S')

In [419]:
# Get rid of the duplicate entries 
turnstiles_df.sort_values(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE_TIME'], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=['C/A', 'UNIT', 'SCP', 'STATION', 'DATE_TIME'], inplace=True)

In [422]:
# get maximum entries per day per turnstile
turnstiles_daily = (turnstiles_df
                        .groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE'],as_index=False)
                        .ENTRIES.first())
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,59 ST,12/01/2019,7292166
1,A002,R051,02-00-00,59 ST,12/02/2019,7293554
2,A002,R051,02-00-00,59 ST,12/03/2019,7295052
3,A002,R051,02-00-00,59 ST,12/04/2019,7296585
4,A002,R051,02-00-00,59 ST,12/05/2019,7298288


In [423]:
# get total entries per day per turnstile                      
turnstiles_daily[['PREV_DATE', 'PREV_ENTRIES']] = (turnstiles_daily
                                                       .groupby(['C/A', 'UNIT', 'SCP', 'STATION'])[['DATE', 'ENTRIES']]
                                                       .shift(1))
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
0,A002,R051,02-00-00,59 ST,12/01/2019,7292166,NaN,NaN
1,A002,R051,02-00-00,59 ST,12/02/2019,7293554,12/01/2019,7292166.0
2,A002,R051,02-00-00,59 ST,12/03/2019,7295052,12/02/2019,7293554.0
3,A002,R051,02-00-00,59 ST,12/04/2019,7296585,12/03/2019,7295052.0
4,A002,R051,02-00-00,59 ST,12/05/2019,7298288,12/04/2019,7296585.0


In [426]:
# Drop the rows for the earliest date in the df
turnstiles_daily.dropna(subset=['PREV_DATE'], axis=0, inplace=True)
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
1,A002,R051,02-00-00,59 ST,12/02/2019,7293554,12/01/2019,7292166.0
2,A002,R051,02-00-00,59 ST,12/03/2019,7295052,12/02/2019,7293554.0
3,A002,R051,02-00-00,59 ST,12/04/2019,7296585,12/03/2019,7295052.0
4,A002,R051,02-00-00,59 ST,12/05/2019,7298288,12/04/2019,7296585.0
5,A002,R051,02-00-00,59 ST,12/06/2019,7299886,12/05/2019,7298288.0


for the `get_daily_counts` function, I set the `max counter` value to 20000:

In [427]:
def get_daily_counts(row, max_counter):
    counter = row['ENTRIES'] - row['PREV_ENTRIES']
    
    if counter < 0:
        counter = -counter  # adjust for "reverse" counter
        
    if counter > max_counter:
        # Maybe counter was reset, so it may make sense to take the minimum
        print(f"ENTRIES: {row['ENTRIES']} <-- {row['PREV_ENTRIES']}")
        counter = min(row['ENTRIES'], row['PREV_ENTRIES'])
        
    if counter > max_counter:
        # If we still get a counter that is too big, set to zero
        return 0
    
    return counter

turnstiles_daily['DAILY_ENTRIES'] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=20000)                                                   

ENTRIES: 1091 <-- 130724.0
ENTRIES: 459077 <-- 85175465.0
ENTRIES: 170526823 <-- 1802952.0
ENTRIES: 458753 <-- 0.0
ENTRIES: 983284 <-- 27934.0
ENTRIES: 582 <-- 3243165.0
ENTRIES: 67120006 <-- 292100.0
ENTRIES: 271492 <-- 67132807.0
ENTRIES: 678 <-- 5270575.0
ENTRIES: 394 <-- 79785.0
ENTRIES: 36579514 <-- 339421.0
ENTRIES: 454 <-- 356766.0
ENTRIES: 1378 <-- 377044.0
ENTRIES: 1052 <-- 13538231.0
ENTRIES: 2526 <-- 6920350.0
ENTRIES: 1892 <-- 1060083.0
ENTRIES: 1690 <-- 5916160.0
ENTRIES: 997 <-- 124114.0
ENTRIES: 131144 <-- 1743463.0
ENTRIES: 458861 <-- 843546.0
ENTRIES: 589824 <-- 866455.0
ENTRIES: 249 <-- 91596.0
ENTRIES: 193 <-- 22403.0
ENTRIES: 1404 <-- 97612.0
ENTRIES: 1928 <-- 444626.0
ENTRIES: 178 <-- 21346.0
ENTRIES: 1196 <-- 116461.0
ENTRIES: 76 <-- 43414.0
ENTRIES: 41 <-- 40491.0
ENTRIES: 2355 <-- 54367.0
ENTRIES: 1353 <-- 69354.0
ENTRIES: 2598 <-- 73791.0
ENTRIES: 80 <-- 385686.0
ENTRIES: 2041 <-- 336934.0
ENTRIES: 0 <-- 23433.0
ENTRIES: 0 <-- 161739.0
ENTRIES: 862 <-- 42103.0


In [428]:
#turnstiles_daily.to_sql('mta_turnstiles_daily', engine)

## MTA Visualizations

In [430]:
#Read in the data for turnstiles
turnstiles_2019 = pd.read_sql('SELECT * FROM mta_turnstiles_daily WHERE [DATE] LIKE "%/2019"', engine)
turnstiles_2021 = pd.read_sql('SELECT * FROM mta_turnstiles_daily WHERE [DATE] LIKE "%/2021"', engine)

#### Creating a daily count of entries by station for 2019 and 2021:

In [431]:
station_daily_2019 = turnstiles_2019.groupby(['STATION', 'DATE'])[['DAILY_ENTRIES']].sum().reset_index()
station_daily_2021 = turnstiles_2021.groupby(['STATION', 'DATE'])[['DAILY_ENTRIES']].sum().reset_index()

In [435]:
station_daily_2019.head(20)

,STATION,DATE,DAILY_ENTRIES
0,1 AV,01/02/2019,0.0
1,1 AV,01/03/2019,0.0
2,1 AV,01/04/2019,0.0
3,1 AV,01/05/2019,0.0
4,1 AV,01/06/2019,0.0
5,1 AV,01/07/2019,0.0
6,1 AV,01/08/2019,0.0
7,1 AV,01/09/2019,504.0
8,1 AV,01/10/2019,531.0
9,1 AV,01/11/2019,547.0


In [436]:
station_daily_2021.head()

,STATION,DATE,DAILY_ENTRIES
0,1 AV,01/01/2021,20612.0
1,1 AV,01/02/2021,22772.0
2,1 AV,01/03/2021,23403.0
3,1 AV,01/04/2021,22795.0
4,1 AV,01/05/2021,22860.0


#### Sorting by the largest daily entries to get the highest-traffic stations:

In [437]:
station_totals_2019 = (station_daily_2019.groupby('STATION')['DAILY_ENTRIES'].sum()
    .reset_index()
    .sort_values('DAILY_ENTRIES', ascending=False))

station_totals_2019.head()

,STATION,DAILY_ENTRIES
315,PATH NEW WTC,60968882.0
279,METS-WILLETS PT,50180728.0
300,NEWARK HM HE,39476119.0
239,HARRISON,38389418.0
184,CITY / BUS,36288482.0


In [438]:
station_totals_2021 = (station_daily_2021.groupby('STATION')['DAILY_ENTRIES'].sum()
    .reset_index()
    .sort_values('DAILY_ENTRIES', ascending=False))

station_totals_2021.head()

,STATION,DAILY_ENTRIES
233,GRD CNTRL-42 ST,93079248.0
226,FULTON ST,70975322.0
315,PATH NEW WTC,64763190.0
192,CORTLANDT ST,56049976.0
376,WTC-CORTLANDT,46853862.0


#### Computing the percentage with respect to 2019 traffic for 2021:

In [439]:
station_totals_2021['CHANGE'] = (station_totals_2021['DAILY_ENTRIES']
                                 .div(station_totals_2019['DAILY_ENTRIES']))
#station_totals_2021['CHANGE'] = station_totals_2021['CHANGE'].abs()

station_totals_2021.head()

,STATION,DAILY_ENTRIES,CHANGE
233,GRD CNTRL-42 ST,93079248.0,6.467254
226,FULTON ST,70975322.0,2.000580
315,PATH NEW WTC,64763190.0,1.062234
192,CORTLANDT ST,56049976.0,4.156297
376,WTC-CORTLANDT,46853862.0,2.890587


#### Adding a `2019_ENTRIES` column to the dataframe, and sorting by percentage of 2019 traffic:

I created the `smallest_change_2021` dataframe in order to select the top 200 stations out of the list of stations that have retained the most traffic from pre-pandemic levels.

In [ ]:
station_totals_2021['2019_ENTRIES'] = station_totals_2019['DAILY_ENTRIES']
smallest_change_2021 = station_totals_2021.reset_index().sort_values(by=['CHANGE'], ascending = False).head(200)
smallest_change_2021.head()

#### Finding the stations from the previous list that have the largest total entries:

In order to get that sweet spot between total volume and retained volume. I want to be able to recommend a selection of ten stations that the city health department should choose for their newest COVID testing and vaccination stations, so I want high volume but also high traffic retention (which implies larger commuter/local usage).

In [ ]:
largest_sorted_2021 = smallest_change_2021.reset_index().sort_values(by=['DAILY_ENTRIES', 'CHANGE'], ascending = False).head(50)
largest_sorted_2021.head(10)

#### EXAMPLE: plotting the highest traffic stations from 2019 relative to their 2021 traffic:

In [ ]:
top_stations=station_totals_2021['station'][:10]

fig = go.Figure(data=[
    go.Bar(name='2019 Entries', 
           marker_color='dimgrey', 
           opacity=0.9, 
           x=top_stations, 
           y=station_totals_2021['2019_entries'][:10]),
    go.Bar(name='2021 Entries', 
           marker_color='firebrick', 
           opacity=0.9, 
           x=top_stations, 
           y=station_totals_2021['daily_entries'][:10])
])
# Change the layout
fig.update_layout(title_text='MTA Usage Pre-Pandemic vs. 2021',
                   barmode='group',
                   hovermode='x unified',
                   xaxis_tickangle=75, 
                   template="simple_white") 
fig.show()

#### By comparison, my analysis shows much higher traffic retention outside of Manhattan

In [ ]:
top_stations=largest_sorted_2021['station'][:10]

fig = go.Figure(data=[
    go.Bar(name='2019 Entries', 
           marker_color='dimgrey', 
           opacity=0.9, 
           x=top_stations, 
           y=largest_sorted_2021['2019_entries'][:10]),
    go.Bar(name='2021 Entries', 
           marker_color='firebrick', 
           opacity=0.9, 
           x=top_stations, 
           y=largest_sorted_2021['daily_entries'][:10])
])
# Change the layout
fig.update_layout(title_text='MTA Usage Pre-Pandemic vs. 2021',
                   barmode='group',
                   hovermode='x unified',
                   xaxis_tickangle=75, 
                   template="simple_white") 
fig.show()